<a href="https://colab.research.google.com/github/junzer0/23-HP-017/blob/main/anc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#데이터 수집 및 전처리

In [ ]:

#******보완사항******
#저장된 stft데이터를 csv파일로 제작

#필요사항 import
import numpy as np
import pandas as pd
import librosa

#오디오 데이터를 stft처리하는 함수
def load_data(file_path, sr=16000, duration=10):
    audio, _ = librosa.load(file_path, sr=sr, duration=duration)
    stft_data = librosa.stft(audio)
    return stft_data

#stft데이터를 csv파일로 변환하는 함수
def create_csv_from_stft(stft_data, output_csv_path, split_duration=0.1):
    num_splits = stft_data.shape[1]

    data_list = []
    for i in range(num_splits):
        magnitude = np.abs(stft_data[:, i])
        phase = np.angle(stft_data[:, i])
        time = i * split_duration
        data_list.append([time, magnitude, phase])

    df = pd.DataFrame(data_list, columns=["time", "magnitude", "phase"])
    df.to_csv(output_csv_path, index=False)

#인풋 파일 경로와 출력 CSV 파일 경로를 지정
input_external_file = 'external_noise.wav'
input_internal_file = 'internal_noise.wav'
external_csv = 'external_noise_stft.csv'
internal_csv = 'internal_noise_stft.csv'

# 데이터 로드 및 STFT 분석을 수행
external_noise_data = load_data('external_noise.wav')
internal_noise_data = load_data('internal_noise.wav')

# STFT 데이터를 CSV 파일로 저장 수행
create_csv_from_stft(external_noise_data, external_csv)
create_csv_from_stft(internal_noise_data, internal_csv)

#저장한 csv파일을 코드에 사용 가능하도록 불러오기
external_usable=pd.read_csv("external.csv")
internal_usable=pd.read_csv("internal.csv")

In [ ]:
#LSTM 모델 구축

In [ ]:
#시각화 및 데이터 전처리 도구 IMPORT

import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np

#딥러닝 모델 제작 툴 라이브러리 IMPORT

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import *

#LSTM 라이브러리 IMPORT

from keras.layers import LSTM

#세부사항 라이브러리 IMPORT

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [ ]:
# Feature를 Scaling

sc = MinMaxScaler(feature_range = (0, 1))

external_noise_data_scaled = sc.fit_transform(external_usable)


In [ ]:


#LSTM 모델 제작

model = Sequential()

#첫번째 LSTM layer 추가

model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1),activation='tanh'))

model.add(Dropout(0.2))  #20퍼센트의 뉴런을 off 한다( 오버피팅을 막기 위해)

#두번째 LSTM layer 추가

model.add(LSTM(units = 50, return_sequences = True,activation='tanh'))

model.add(Dropout(0.2))  #20퍼센트의 뉴런을 off 한다( 오버피팅을 막기 위해)

#세번째 LSTM layer 추가

model.add(LSTM(units = 50, return_sequences = True,activation='tanh'))

model.add(Dropout(0.2))  #20퍼센트의 뉴런을 off 한다( 오버피팅을 막기 위해)

#네번째 LSTM layer 추가

model.add(LSTM(units = 50))

model.add(Dropout(0.2))  #20퍼센트의 뉴런을 off 한다( 오버피팅을 막기 위해)

#output layer 설정

model.add(Dense(units = 1))

#RNN 컴파일링

model.compile(optimizer='adam',loss='mean_squared_error')


#모델 적용 및 훈련 시작

model.fit(external_noise_data_scaled, internal_usable, epochs = 100, batch_size = 32)


In [ ]:
#예측 모델 생성 및 학습
#학습은 이미 완료된 모델을 사용하므로 또 필요 X

anc_model = model

def apply_anc(external_noise_data_scaled, anc_model):
    predicted_internal_noise = anc_model.predict(external_noise_data_scaled)
    return predicted_internal_noise

predicted_internal_noise = apply_anc(external_noise_data_scaled, anc_model)



In [ ]:
# 예측된 내부 소음과 실제 내부 소음을 비교(시각화)


#시각화

plt.plot(internal_usable.loc[0:, 'time'],internal_usable, color = 'red', label = 'Real internal noise')

plt.plot(internal_usable.loc[0:, 'time'],predicted_internal_noise, color = 'blue', label = 'Predicted internal noise')

plt.xticks(np.arange(0,len(internal_usable),50))

plt.title('Prediction for internal_noise')

plt.xlabel('Time')

plt.ylabel('internal noise')

plt.legend()

plt.show()


In [ ]:
# 내부 소음 차이 값 도출
denoised_internal_noise = internal_noise_data - predicted_internal_noise # LSTM으로 돌려서 차이값 줄여보는 방안도 있을듯.

# ISTFT 변환
def istft(stft_matrix):
    return librosa.istft(stft_matrix)

# 예측 내부 소음의 역위상을 시간 도메인 신호로 변환
anc_audio = istft(predicted_internal_noise)

# 결과 저장
sf.write('anc_audio.wav', anc_audio, 16000)